In this notebook we will perform classification on the Human activity recognition data
which you can find in this [link](https://www.kaggle.com/datasets/uciml/human-activity-recognition-with-smartphones). The database was built from the recordings of 30 study participants performing activities of daily living (ADL) while carrying a waist-mounted smartphone with embedded inertial sensors. The objective is to classify activities into one of the six activities performed. For this task you need to first download the data from aforementioned link and keep it in the `data` directory inside tutorials. Also, download the pandas library via `pip install pandas`

In [1]:
import pdb
import numpy as np
import pandas as pd 
import sys
sys.path.insert(0, '../')
from reservoirpy.nodes import Reservoir, Input, LinearRegression, RidgeRegression, ElasticNet, Lasso
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from reservoirpy.datasets import mackey_glass
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score

In [2]:
# First read the data using pandas and convert into numpy format
train = pd.read_csv("data/train.csv")
print(train.head())
X_train= train.iloc[:,:-2].to_numpy()
Y_train= train.iloc[:,-1]
test= pd.read_csv("data/test.csv")
X_test=test.iloc[:,:-2].to_numpy()
Y_test= test.iloc[:,-1]

   tBodyAcc-mean()-X  tBodyAcc-mean()-Y  tBodyAcc-mean()-Z  tBodyAcc-std()-X  \
0           0.288585          -0.020294          -0.132905         -0.995279   
1           0.278419          -0.016411          -0.123520         -0.998245   
2           0.279653          -0.019467          -0.113462         -0.995380   
3           0.279174          -0.026201          -0.123283         -0.996091   
4           0.276629          -0.016570          -0.115362         -0.998139   

   tBodyAcc-std()-Y  tBodyAcc-std()-Z  tBodyAcc-mad()-X  tBodyAcc-mad()-Y  \
0         -0.983111         -0.913526         -0.995112         -0.983185   
1         -0.975300         -0.960322         -0.998807         -0.974914   
2         -0.967187         -0.978944         -0.996520         -0.963668   
3         -0.983403         -0.990675         -0.997099         -0.982750   
4         -0.980817         -0.990482         -0.998321         -0.979672   

   tBodyAcc-mad()-Z  tBodyAcc-max()-X  ...  fBodyBodyGyr

In [3]:
X_train = 2 * (X_train - X_train.min()) / (X_train.max() - X_train.min()) - 1
X_test = 2 * (X_test - X_test.min()) / (X_test.max() - X_test.min()) - 1

The labels i.e. Y_train and Y_test are in string format, thus, in order to convert them
into integers we will have to create a mapping between the string labels and their respective
class integers.

In [4]:
label2id = {"SITTING":0, "WALKING_DOWNSTAIRS":1,
"WALKING_UPSTAIRS":2, "STANDING":3, "WALKING":4,
"LAYING":5
}
Y_train = np.array([label2id[Y_train[i]] for i in range(len(Y_train))])
Y_test = np.array([label2id[Y_test[i]] for i in range(len(Y_test))])

Now, the target labels are in integer format, howvwer since we are performing regression, 
the output labels need to be in vectors. Thus we perform one-hot encoding on the training labels

In [5]:
print(f" Classes: {set(Y_train)} ")
Y_train_onehot = np.zeros((Y_train.shape[0], len(set(Y_train))))
for i in range(len(Y_train)):
	Y_train_onehot[i, Y_train[i]] = 1
Y_train = Y_train_onehot

 Classes: {0, 1, 2, 3, 4, 5} 


Now, we will create the model

In [7]:
source = Input()
reservoir = Reservoir(500, sr=0.9, lr=0.1)
readout = Lasso(alpha=0.001)
model = source >> reservoir >> readout

We train the model by taking the last state from the reservoir node and storing it into lists.
Finally we call our fit method on the RidgeRegression readout node on the last satetes and target labels.

In [8]:
states_train = []
for x in X_train:
    states = reservoir.run(x, reset=True)
    states_train.append(states[-1, np.newaxis])
readout.fit(np.array(states_train).squeeze(), np.array(Y_train).squeeze())

Running Reservoir-1: 100%|██████████| 1/1 [00:00<00:00, 2304.56it/s]


Running Reservoir-1: 100%|██████████| 1/1 [00:00<00:00, 1661.11it/s]


Running Reservoir-1: 100%|██████████| 1/1 [00:00<00:00, 2702.52it/s]


Running Reservoir-1: 100%|██████████| 1/1 [00:00<00:00, 3509.88it/s]


Running Reservoir-1: 100%|██████████| 1/1 [00:00<00:00, 2890.63it/s]


...

Running Reservoir-1: 100%|██████████| 1/1 [00:00<00:00, 2416.07it/s]


Running Reservoir-1: 100%|██████████| 1/1 [00:00<00:00, 2462.89it/s]


Running Reservoir-1: 100%|██████████| 1/1 [00:00<00:00, 802.58it/s]


Running Reservoir-1: 100%|██████████| 1/1 [00:00<00:00, 1620.67it/s]IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



'Lasso-0': Lasso(f=<function lasso at 0x7f9e2a55ea60>, in=500, out=6)

Finally, we perform prediction on the test data and measure the accuracy.

In [9]:
states_test = []
for x in X_test:
    states = reservoir.run(x, reset=True)
    states_test.append(states[-1, np.newaxis])

Running Reservoir-1: 100%|██████████| 1/1 [00:00<00:00, 389.41it/s]


Running Reservoir-1: 100%|██████████| 1/1 [00:00<00:00, 3650.40it/s]


Running Reservoir-1: 100%|██████████| 1/1 [00:00<00:00, 2993.79it/s]


Running Reservoir-1: 100%|██████████| 1/1 [00:00<00:00, 2644.58it/s]


Running Reservoir-1: 100%|██████████| 1/1 [00:00<00:00, 2725.34it/s]


...

Running Reservoir-1: 100%|██████████| 1/1 [00:00<00:00, 4165.15it/s]


Running Reservoir-1: 100%|██████████| 1/1 [00:00<00:00, 2493.64it/s]


Running Reservoir-1: 100%|██████████| 1/1 [00:00<00:00, 2563.76it/s]


Running Reservoir-1: 100%|██████████| 1/1 [00:00<00:00, 1669.71it/s]


Running Reservoir-1: 100%|██████████| 1/1 [00:00<00:00, 2688.66it/s]


In [10]:
y_pred = readout.run(np.array(states_test).squeeze())
Y_pred_class = np.argmax(y_pred, axis=1)


Running Lasso-0: 100%|██████████| 2947/2947 [00:00<00:00, 20636.55it/s]


In [11]:
score = accuracy_score(Y_test, Y_pred_class)
print("Accuracy: ", f"{score * 100:.3f} %")

Accuracy:  85.409 %


### Comparison with Native Ridge Node

In [87]:
from reservoirpy.nodes import Ridge
source = Input()
reservoir = Reservoir(500, sr=0.9, lr=0.1)
readout = Ridge(ridge=0.027598)
model_native = source >> reservoir >> readout

In [88]:
X_train_native = [X_train[i] for i in range(X_train.shape[0])]
X_test_native = [X_test[i] for i in range(X_test.shape[0])]

In [89]:
states_train = []
for x in X_train_native:
    states = reservoir.run(x, reset=True)
    states_train.append(states[-1, np.newaxis])
readout.fit(np.array(states_train).squeeze(), np.array(Y_train).squeeze())

Running Reservoir-13: 100%|██████████| 1/1 [00:00<00:00, 3775.25it/s]


Running Reservoir-13: 100%|██████████| 1/1 [00:00<00:00, 1176.52it/s]


Running Reservoir-13: 100%|██████████| 1/1 [00:00<00:00, 2611.65it/s]


Running Reservoir-13: 100%|██████████| 1/1 [00:00<00:00, 2531.26it/s]


Running Reservoir-13: 100%|██████████| 1/1 [00:00<00:00, 2647.92it/s]
IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

Running Reservoir-13: 100%|██████████| 1/1 [00:00<00:00, 1600.27it/s]


...

Running Reservoir-13: 100%|██████████| 1/1 [00:00<00:00, 3066.01it/s]


Running Reservoir-13: 100%|██████████| 1/1 [00:00<00:00, 1023.50it/s]


Running Reservoir-13: 100%|██████████| 1/1 [00:00<00:00, 2545.09it/s]


Running Reservoir-13: 100%|██████████| 1/1 [00:00<00:00, 1754.20it/s]


Running Reservoir-13: 100%|██████████| 1/1 [00:00<00:00, 1631.39it/s]IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [90]:
states_test = []
for x in X_test_native:
    states = reservoir.run(x, reset=True)
    states_test.append(states[-1, np.newaxis])

Running Reservoir-13: 100%|██████████| 1/1 [00:00<00:00, 3666.35it/s]


Running Reservoir-13: 100%|██████████| 1/1 [00:00<00:00, 1366.22it/s]


Running Reservoir-13: 100%|██████████| 1/1 [00:00<00:00, 2543.54it/s]


Running Reservoir-13: 100%|██████████| 1/1 [00:00<00:00, 2916.76it/s]


Running Reservoir-13: 100%|██████████| 1/1 [00:00<00:00, 2475.98it/s]
IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)


Running Reservoir-13: 100%|██████████| 1/1 [00:00<00:00, 612.75it/s]


Running Reservoir-13: 100%|██████████| 1/1 [00:00<00:00, 2882.68it/s]


Running Reservoir-13: 100%|██████████| 1/1 [00:00<00:00, 909.83it/s]


Running Reservoir-13: 100%|██████████| 1/1 [00:00<00:00, 1969.16it/s]


Running Reservoir-13: 100%|██████████| 1/1 [00:00<00:00, 667.56it/s]IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [91]:
y_pred = readout.run(np.array(states_test).squeeze())
Y_pred_class = np.argmax(y_pred, axis=1)

Running Ridge-6: 100%|██████████| 2947/2947 [00:00<00:00, 60810.34it/s]


In [92]:
score = accuracy_score(Y_test, Y_pred_class)
print("Accuracy: ", f"{score * 100:.3f} %")

Accuracy:  92.535 %


### Model with only Scikit RidgeRegression Node

In [32]:
source = Input()
readout = ElasticNet(alpha=0.001)
model = source >> readout

In [33]:
readout.fit(X_train, Y_train)

/Users/deep/opt/miniconda3/envs/pytorch2/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:529: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 5.98143835898577, tolerance: 0.1061054951033727
  model = cd_fast.enet_coordinate_descent(
/Users/deep/opt/miniconda3/envs/pytorch2/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:529: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 3.770738345161263, tolerance: 0.08537644178454872
  model = cd_fast.enet_coordinate_descent(
/Users/deep/opt/miniconda3/envs/pytorch2/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:529: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 9.241450552762714, tolerance: 0.09163992110990155
  model = cd_fast.enet_coordinate_descent(
/Users/deep/opt/miniconda3/

'ElasticNet-3': ElasticNet(f=<function elastic_net at 0x7f9e2a55e9d0>, in=561, out=6)

In [34]:
y_pred = readout.run(X_test)
Y_pred_class = np.argmax(y_pred, axis=1)

Running ElasticNet-3: 100%|██████████| 2947/2947 [00:00<00:00, 19655.46it/s]


In [35]:
score = accuracy_score(Y_test, Y_pred_class)
print("Accuracy: ", f"{score * 100:.3f} %")

Accuracy:  95.792 %
